In [4]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [5]:
#read new combined cleaned dataset
df = pd.read_csv("..\Project 1\WSB Data\wsb.csv", index_col=0)


In [6]:
#sort by num_mentions rolled up into ticker
df_num_mentions =df.sort_values(["num_mentions","ticker"],ascending=False).groupby("ticker")
df_num_mentions = df.groupby('ticker')['num_mentions'].sum().sort_values(ascending=False)
df_num_mentions.head(20)

#df_num_mentions.to_csv('test.csv')

ticker
GME     122643
BB       41669
AMC      38061
PLTR     20785
CLOV     15889
WISH     10965
SPY      10823
RKT      10520
TSLA     10194
CLNE      8252
NOK       8133
UWMC      6239
AMD       5620
TLRY      5362
SNDL      5133
AAPL      5040
WKHS      4565
MVIS      4465
NIO       4198
ALL       4063
Name: num_mentions, dtype: int64

In [7]:
#sort by pct_mentions rolled up into ticker
df_pct_mentions = df.sort_values(["pct_mentions","ticker"],ascending=False).groupby("ticker")
df_pct_mentions = df.groupby('ticker')['pct_mentions'].mean().sort_values(ascending=False)
df_pct_mentions.head(20)

#df_pct_mentions.to_csv('test.csv')

ticker
GME     0.165911
AMC     0.058821
NAPA    0.056667
BB      0.049852
CLOV    0.045179
PLTR    0.039604
SPY     0.037637
IRNT    0.035556
WISH    0.027882
SDC     0.026587
DOLE    0.025000
TSLA    0.022178
HOOD    0.021477
OPAD    0.020000
MVIS    0.018916
TMC     0.018611
MNMD    0.017564
CLNE    0.016797
RKT     0.016405
MVST    0.015000
Name: pct_mentions, dtype: float64

In [8]:
#sort by pos_count rolled up into ticker
df_pos_count =df.sort_values(["pos_count","ticker"],ascending=False).groupby("ticker")
df_pos_count = df.groupby('ticker')['pos_count'].sum().sort_values(ascending=False)
df_pos_count.head(20)

#df_pos_count.to_csv('test.csv')

ticker
GME     57609
BB      17018
AMC     16999
PLTR     8497
WISH     7928
CLOV     5964
RKT      5233
TSLA     4434
SPY      4033
CLNE     3651
UWMC     3128
NOK      3075
AMD      2974
TLRY     2351
MVIS     1995
WKHS     1968
APHA     1941
SNDL     1859
CLF      1763
NIO      1643
Name: pos_count, dtype: int64

In [9]:
#sort by neg_count rolled up into ticker
df_neg_count =df.sort_values(["neg_count","ticker"],ascending=False).groupby("ticker")
df_neg_count = df.groupby('ticker')['neg_count'].sum().sort_values(ascending=False)
df_neg_count.head(20)

#df_neg_count.to_csv('test.csv')

ticker
GME     32026
AMC      8611
BB       7202
PLTR     4758
SPY      3128
CLOV     2654
TSLA     2554
RKT      2518
ALL      1552
NOK      1478
NIO      1270
CLNE     1242
UWMC     1211
AMD      1189
WISH     1185
AAPL      976
SNDL      967
BABA      934
MVIS      906
TLRY      840
Name: neg_count, dtype: int64